In [65]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from random import randint, choice, choices
from math import sqrt


In [66]:
# Load the images
def load_images(DIR):
    fn = [os.path.join(DIR, f) for f in sorted(os.listdir(DIR)) if f.endswith(".JPG")]
    return np.array([cv2.imread(f) for f in fn]), np.array(
        [cv2.imread(f, 0) for f in fn]
    )

In [67]:
# Harris Corner Detection
def harris_corner_detector(img, colimg, k=0.04, window_size=3, threshold=0.0025):
    # Gaussian Blur over the image
    img = cv2.GaussianBlur(img, (window_size, window_size), 0)

    # Compute the gradients
    Ix = cv2.Sobel(img, cv2.CV_64F, dx=0, dy=1, ksize=window_size)
    Iy = cv2.Sobel(img, cv2.CV_64F, dx=1, dy=0, ksize=window_size)

    # Compute the products of the gradients
    IxIy = Ix * Iy
    Ix2 = Ix**2
    Iy2 = Iy**2

    # Compute the sums of the products of the gradients
    S2x = cv2.GaussianBlur(Ix2, (window_size, window_size), 0)
    S2y = cv2.GaussianBlur(Iy2, (window_size, window_size), 0)
    Sxy = cv2.GaussianBlur(IxIy, (window_size, window_size), 0)

    # Harris Corner Response
    det = (S2x * S2y) - (Sxy**2)
    trace = S2x + S2y

    R = det - k * (trace**2)

    # Normalize
    R /= R.max()

    # Dilate
    # Rd = cv2.dilate(R, None)
    Rd = R

    # Thresholding
    R[Rd > threshold] = 255

    # Non Max Suppression
    R[Rd <= threshold] = 0

    return R

In [68]:
def NCC(f, g):
    f_hat = f / np.linalg.norm(f)
    g_hat = g / np.linalg.norm(g)

    ncc = np.sum(f_hat * g_hat)
    return ncc


In [74]:
def showi(images):
    for i, im in enumerate(images):
        cv2.imshow(str(i), im)
    if cv2.waitKey(0) & 0xFF == ord("q"):
        cv2.destroyAllWindows()

In [70]:
DIR = "DanaHallWay1"  # --> k=0.0025
k = 0.0025
ncc_thresh = 0.9995
# DIR = "DanaOffice" #--> k=0.01 (?)
# k = 0.01
# ncc_thresh = 0.95
col, gray = load_images(DIR)

harris = []
for i in range(len(col)):
    R1 = harris_corner_detector(gray[i], col[i], threshold=k)
    R2 = harris_corner_detector(gray[i + 1], col[i + 1], threshold=k)
    break

corner1 = np.where(R1 == 255)
corner2 = np.where(R2 == 255)

# List of coordinates of corners
c1coords = list(zip(corner1[0], corner1[1]))
c2coords = list(zip(corner2[0], corner2[1]))

# samples = 600
# c1coords = choices(c1coords, k=samples)
# c2coords = choices(c2coords, k=samples)

# Window size and padding for NCC
window_size = 5
pad = int((window_size - 1) / 2)

# Pad gray images
g1_pad = np.pad(gray[0], pad, "constant", constant_values=0)
g2_pad = np.pad(gray[1], pad, "constant", constant_values=0)

# Perform NCC
# - Compare every corner detected in image 1 with every corner detected in image 2
# - Store the pairs of corners with the highest correlation value
corners = {}
for i, corner1 in enumerate(c1coords):
    # Image patch around corner 1
    x1 = max(corner1[0] - pad, 0)
    x2 = max(corner1[0] + pad + 1, window_size)
    y1 = max(corner1[1] - pad, 0)
    y2 = max(corner1[1] + pad + 1, window_size)
    patch1 = g1_pad[
        x1 : x2,
        y1 : y2,
    ]

    maxNCC = -1
    coords = None
    for j, corner2 in enumerate(c2coords):
        print(f"i={i}/{len(c1coords)} j={j}/{len(c2coords)}", end="\r")

        # Image patch around corner 2
        x1 = max(corner2[0] - pad, 0)
        x2 = max(corner2[0] + pad + 1, window_size)
        y1 = max(corner2[1] - pad, 0)
        y2 = max(corner2[1] + pad + 1, window_size)
        patch2 = g2_pad[
            x1 : x2,
            y1 : y2,
        ]
        # Calculate NCC using image patches
        ncc = NCC(patch1, patch2)
        # If this NCC is the new max, store it and the coords of the corner
        if ncc > maxNCC:
            maxNCC = ncc
            coords = corner2[0], corner2[1]

    # Break earlier for testing
    # if i > 100:
    #     break

    # Threshold
    if maxNCC < ncc_thresh:
        continue

    # Store corner pair with highest NCC
    corners[(corner1[0], corner1[1])] = coords
    # Color the corners in the images
    # col[0][corner1[0], corner1[1]] = [0, 0, 255]
    # col[1][coords[0], coords[1]] = [0, 0, 255]


In [75]:
# corners: dict[tuple[int, int]: tuple[int, int]]
def drawCorrelationLines(image1, image2, corners):
    # Concatenate the two images
    vis = np.concatenate((image1, image2), axis=1)

    # Draw lines between correlated corners
    for key, value in corners.items():
        r = randint(0, 255)
        g = randint(0, 255)
        b = randint(0, 255)
        start_y, start_x = key
        end_y, end_x = value
        end_x = int(end_x + vis.shape[1]/2)
        end = (end_x, end_y)
        start = (start_x, start_y)
        cv2.line(vis, start, end, (b, g, r), 1)

    showi([vis])

In [76]:
drawCorrelationLines(col[0], col[1], corners)

In [108]:
k = 100
N = 4

inliers = {}
outliers = {}
max_inliers = 0

# TODO: This is kinda arbitrary - trial and error maybe
dist_threshold = 4

for i in range(k):
    # print(f"===== Iteration #{i + 1}")
    points1 = []
    points2 = []

    set_inliers = {}
    set_outliers = {}

    num_inliers = 0
    num_outliers = 0

    # Sample the minimal number of points needed to estimate a homography (4 pts)
    for _ in range(N):
        pt1 = choice(list(corners.keys()))
        pt2 = list(corners[pt1])
        pt1 = list(pt1)
        points1.append(pt1)
        points2.append(pt2)
    points1 = np.asarray(points1)
    points2 = np.asarray(points2)

    # Compute homography matrix using these points
    h, status = cv2.findHomography(points1, points2)
    try:
        h_inv = np.linalg.pinv(h)
    except:
        continue

    for corner1, corner2 in corners.items():
        # Estimate point using homography
        pt1 = np.array([corner1[0], corner1[1], 1])
        pt2 = np.array([corner2[0], corner2[1], 1])
        res = np.matmul(h, pt1)
        res = (res[:2]/res[2]).astype(int)
        dist = np.linalg.norm(res-corner2)

        # Check if outlier
        if dist > dist_threshold:
            set_outliers[tuple(corner1)] = tuple(corner2)
            num_outliers += 1
            continue

        # Store inlier
        set_inliers[tuple(corner1)] = tuple(corner2)
        num_inliers += 1

    # Check if this homography produced the new largest set of inliers
    if num_inliers > max_inliers:
        max_inliers = num_inliers
        inliers = set_inliers
        outliers = set_outliers

print(max_inliers, inliers)


60 {(4, 364): (7, 252), (5, 364): (7, 252), (6, 363): (8, 251), (15, 441): (23, 324), (17, 441): (24, 324), (17, 442): (24, 325), (18, 441): (25, 324), (18, 442): (25, 325), (19, 441): (26, 324), (32, 487): (41, 364), (33, 441): (39, 324), (34, 441): (40, 324), (34, 442): (40, 325), (35, 441): (40, 325), (35, 442): (41, 324), (36, 441): (41, 325), (36, 442): (42, 324), (37, 441): (42, 325), (37, 442): (43, 324), (38, 441): (43, 325), (38, 442): (44, 325), (65, 197): (59, 81), (66, 196): (60, 80), (99, 175): (94, 58), (150, 148): (149, 28), (153, 182): (152, 66), (153, 183): (152, 67), (154, 182): (153, 66), (154, 183): (153, 67), (154, 184): (153, 68), (155, 159): (154, 40), (159, 155): (159, 36), (160, 154): (160, 35), (160, 155): (159, 36), (161, 468): (159, 348), (161, 469): (159, 348), (162, 467): (160, 348), (170, 485): (167, 364), (171, 455): (168, 337), (171, 467): (168, 348), (171, 468): (168, 349), (172, 462): (169, 344), (172, 463): (169, 343), (173, 462): (170, 343), (173, 4

In [109]:
# Compute least-squares homography from ALL the inliers in the largest set of inliers
points1 = np.array([*inliers.keys()])
points2 = np.array([*inliers.values()])
H, status = cv2.findHomography(points1, points2)

In [110]:
print(len(inliers))
drawCorrelationLines(col[0], col[1], inliers)

60


In [103]:
print(len(outliers))
drawCorrelationLines(col[0], col[1], outliers)

54


In [ ]:
cornersH = {}

im1pt1, im1pt2 = choices(list(inliers.keys()), k=2)
im2pt1, im2pt2 = inliers[im1pt1], inliers[im1pt2]

sticher=cv2.Stitcher.create()
result = sticher.stitch((col[0], col[1], col[2]))

showi([col[0], col[1], result[1]])
    